# Timings

How the participants progress through the experiment.

## 1 Setup

Flags

In [1]:
SAVE_FIGURES = False

First, setup the database

In [2]:
import os, sys
sys.path.insert(1, os.path.abspath('../..'))
import analysis

FIG = os.path.join(os.path.abspath(os.path.curdir), '{}.png')
DB_NAME = 'spreadr_' + os.path.split(os.path.abspath(os.path.curdir))[1]
analysis.setup(DB_NAME)
print('Database:', DB_NAME)

Database: spreadr_pilot_8b


Then import all we need for the analysis

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
from django.db.models import Count

from gists.models import Sentence, Profile, Questionnaire, GistsConfiguration

config = GistsConfiguration.get_solo()

## 2 Registration ➙ First training sentence

$= time(homeInstructions) + t(expInstructions) + t(trainingSentence)$

Get non-staff profiles who completed the experiment

In [4]:
profiles = Profile.objects\
    .annotate(Count('sentences'))\
    .filter(sentences__count=config.experiment_work + config.training_work,
            user__is_staff=False)

Get mean time from profile creation to first training sentence creation

In [5]:
registration_to_first_training = np.array(
    [profile.sentences.filter(bucket='training').order_by('created').first().created
     - profile.created for profile in profiles])\
    .mean()
print('Registration ➙ First training sentence avg: {}'.format(registration_to_first_training))

Registration ➙ First training sentence avg: 0:01:59.900000


## 3 First training sentence ➙ Last training sentence

$= t(training) - t(trainingSentence)$

In [6]:
first_to_last_training = np.array(
    [profile.sentences.filter(bucket='training').order_by('-created').first().created
     - profile.sentences.filter(bucket='training').order_by('created').first().created
     for profile in profiles])\
    .mean()
print('First training sentence ➙ Last training sentence avg: {}'.format(first_to_last_training))
training_sentence = first_to_last_training / (config.training_work - 1)
print('Training sentence avg: {}'.format(training_sentence))
training = config.training_work * training_sentence
print('Training avg: {}'.format(training))

First training sentence ➙ Last training sentence avg: 0:00:32.900000
Training sentence avg: 0:00:32.900000
Training avg: 0:01:05.800000


## 4 Last training sentence ➙ Questionnaire

$= t(questionnaire)$

In [7]:
#questionnaire = np.array(
#    [profile.questionnaire.created
#     - profile.sentences.filter(bucket='training').order_by('-created').first().created
#     for profile in profiles])\
#    .mean()
#print('Questionnaire avg: {}'.format(questionnaire))

## 5 Questionnaire ➙ First exp sentence

$= t(backToExp) + t(expSentence)$

In [8]:
#questionnaire_to_first_exp = np.array(
#    [profile.sentences.filter(bucket='experiment').order_by('created').first().created
#     - profile.questionnaire.created
#     for profile in profiles])\
#    .mean()
#print('Questionnaire ➙ First exp sentence avg: {}'.format(questionnaire_to_first_exp))

## 6 First exp sentence ➙ Last exp sentence

$= t(exp) - t(expSentence)$

In [9]:
first_to_last_exp = np.array(
    [profile.sentences.filter(bucket='experiment').order_by('-created').first().created
     - profile.sentences.filter(bucket='experiment').order_by('created').first().created
     for profile in profiles])\
    .mean()
print('First exp sentence ➙ Last exp sentence avg: {}'.format(first_to_last_exp))
exp_sentence = first_to_last_exp / (config.experiment_work - 1)
print('Exp sentence avg: {}'.format(exp_sentence))
exp = config.experiment_work * exp_sentence
print('Exp avg: {}'.format(exp))

First exp sentence ➙ Last exp sentence avg: 0:02:59.300000
Exp sentence avg: 0:00:35.860000
Exp avg: 0:03:35.160000


## 7 Registration ➙ Last exp sentence

$= t(total)$

In [10]:
total = np.array(
    [profile.sentences.filter(bucket='experiment').order_by('-created').first().created
     - profile.created
     for profile in profiles])\
    .mean()
print('Total avg: {}'.format(total))

Total avg: 0:06:16.100000


## 8 Summary

In [11]:
def pretty_print(name, timedelta):
    timestr = '{}'.format(timedelta).split('.')[0]
    print(name + ': {0:>{width}}'
          .format(timestr, width=58 - len(name)))

pretty_print('Intro (= home instructions + exp instructions)',
             registration_to_first_training - training_sentence)
pretty_print('Training', training)
pretty_print('Training sentence', training_sentence)
#pretty_print('Questionnaire', questionnaire)
#pretty_print('Back to exp', questionnaire_to_first_exp - exp_sentence)
pretty_print('Exp', exp)
pretty_print('Exp sentence', exp_sentence)
all_sentence = ((first_to_last_training + first_to_last_exp)
                / (config.experiment_work + config.training_work - 2))
pretty_print('All sentence', all_sentence)
pretty_print('Total', total)

Intro (= home instructions + exp instructions):      0:01:27
Training:                                            0:01:05
Training sentence:                                   0:00:32
Exp:                                                 0:03:35
Exp sentence:                                        0:00:35
All sentence:                                        0:00:35
Total:                                               0:06:16
